In [1]:
import sys
sys.path.append('../ry')
from libry import *
from numpy import *

In [2]:
# load and display a configuration
K = Config()
K.addFile('../test/boxProblem.g')
D = K.view()
K.getFrameNames()

['base',
 'box',
 'boxBo',
 'boxLe',
 'boxBa',
 'ballR',
 'block',
 'target',
 'boxBa>target']

In [3]:
# create a standard path optimization problem with 4 phases, each of .2 seconds, and 10 steps
komo = K.komo_path(5., 10, .2)

In [4]:
# create tau-DOFs (time intervals) in the optimization problem and add respective constraints
komo.addTimeOptimization()

In [5]:
# take one object, and first 'switch' it to become freely movable without costs,
# then, starting at phase .7, make it dynamic (imposing NE equations)
obj = 'ballR'
#komo.addSwitch_magic(.0, 'base', obj)
komo.makeObjectsFree(["ballR"]);

In [6]:
# (old) a way to constrain the object position to be 'before' the box before phase 0.2
#komo.addObjective(time=[0., .2], type=OT.ineq, feature=FS.position, frames=[obj], scaleTrans=[[0.,1.,0.]], target=[1.])

In [7]:
# objectives for collision (permanent) and target (at phase-time 4.)
komo.addObjective(type=OT.sos, feature=FS.accumulatedCollisions)
komo.addObjective(time=[5.], type=OT.eq, feature=FS.distance, frames=['target', obj], scale=[3e1])

In [8]:
# simple structure of this solution: bounce on the bottom of the box at times 1, 2, and 3
komo.addSwitch_dynamicTrans(1., -1., 'base', obj)
komo.addInteraction_elasticBounce(2., "boxBo", obj, .8)
komo.addInteraction_elasticBounce(3., "boxBo", obj, .8)
komo.addInteraction_elasticBounce(4., "boxBo", obj, .8)

In [9]:
komo.optimize()

In [10]:
komoView = komo.view()

In [11]:
komo.display()

In [12]:
# read out solution: the full frame path, the tau path (time optimization), list of interaction forces
obj_path = komo.getPathFrames([obj])
tau_path = komo.getPathTau()
forces = komo.getForceInteractions()
forces

[{'at': 19,
  'force': [-4.2457956389799766e-08,
   0.11921128879253202,
   -0.39657575028782255],
  'from': 'boxBo',
  'poa': [0.3306762853441486, -1.2713906082562505, 0.6483517775042474],
  'to': 'ballR'},
 {'at': 29,
  'force': [-4.392341620016822e-07,
   4.284251878299415e-07,
   -0.34991746320870865],
  'from': 'boxBo',
  'poa': [0.245511428940449, -0.28840914872264234, 0.6499915999868081],
  'to': 'ballR'},
 {'at': 39,
  'force': [1.856959661519155e-07,
   -1.1706101479336869e-07,
   -0.294637829049629],
  'from': 'boxBo',
  'poa': [0.17024861134681268, 0.55647045709132, 0.6499807799482432],
  'to': 'ballR'}]

Conversion from steps to phase and time:
```
phase=0 is the FIXED initial configuration and corresponds to step=-1
step=0 is the first configuration s.t. optimization
\tau(s) gives the time between step s-1 and step s
phase(step s) = (s+1)/stepsPerPhase
time(step s) = sum i=0:s tau(i)
```

In [13]:
D=0
komoView=0
komo=0
K=0